In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import utils
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.io import loadmat
import numpy as np
import scipy.io as sio

In [2]:

X=loadmat('/content/PaviaU.mat')['paviaU']
y=loadmat('/content/PaviaU_gt.mat')['paviaU_gt']


In [3]:
dataset='PU'
test_ratio=0.2
pixelsize=25

In [4]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [5]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [6]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [7]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [8]:
X.shape, y.shape

((610, 340, 103), (610, 340))

In [9]:
X,pca=applyPCA(X,numComponents=9)
X.shape

(610, 340, 9)

In [10]:
X, y = createImageCubes(X, y, windowSize=25)

X.shape, y.shape

((42776, 25, 25, 9), (42776,))

In [11]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((34220, 25, 25, 9), (8556, 25, 25, 9), (34220,), (8556,))

In [12]:
Xtrain, Xvalid, ytrain, yvalid = splitTrainTestSet(Xtrain, ytrain, 0.1)

Xtrain.shape, Xvalid.shape, ytrain.shape, yvalid.shape

((30798, 25, 25, 9), (3422, 25, 25, 9), (30798,), (3422,))

In [14]:
Xtrain = Xtrain.reshape(-1, pixelsize, pixelsize, 9, 1)
Xtrain.shape

(30798, 25, 25, 9, 1)

In [15]:
ytrain = utils.to_categorical(ytrain)
ytrain.shape

(30798, 9)

In [16]:
Xvalid = Xvalid.reshape(-1, pixelsize, pixelsize, 9, 1)
Xvalid.shape

(3422, 25, 25, 9, 1)

In [18]:
yvalid=utils.to_categorical(yvalid)
yvalid.shape

(3422, 9)

In [23]:
input_layer=Input((25,25,9,1))
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)
conv3d_shape = conv_layer3.shape
conv_layer3 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer3)
conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer3)
flatten_layer = Flatten()(conv_layer4)
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=9, activation='softmax')(dense_layer2)

(None, 21, 21, 1, 32)


In [24]:
model=Model(inputs=input_layer,outputs=output_layer)


In [25]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 25, 25, 9, 1)]    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 23, 23, 3, 8)      512       
                                                                 
 conv3d_10 (Conv3D)          (None, 21, 21, 1, 32)     6944      
                                                                 
 reshape (Reshape)           (None, 21, 21, 32)        0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 23104)             0         
                                                                 
 dense (Dense)               (None, 256)               591488

In [27]:
adam=Adam(learning_rate=0.01)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
history=model.fit(x=Xtrain,y=ytrain,validation_data=(Xvalid,yvalid),batch_size=256,epochs=10)

Epoch 1/10
121/121 [==============================] - 11s 28ms/step - loss: 1.6370 - accuracy: 0.6219 - val_loss: 0.4499 - val_accuracy: 0.8194
Epoch 2/10
121/121 [==============================] - 2s 19ms/step - loss: 0.4452 - accuracy: 0.8314 - val_loss: 0.3093 - val_accuracy: 0.8749
Epoch 3/10
121/121 [==============================] - 2s 20ms/step - loss: 0.3115 - accuracy: 0.8868 - val_loss: 0.1592 - val_accuracy: 0.9416
Epoch 4/10
121/121 [==============================] - 2s 21ms/step - loss: 0.2126 - accuracy: 0.9242 - val_loss: 0.1280 - val_accuracy: 0.9568
Epoch 5/10
121/121 [==============================] - 2s 20ms/step - loss: 0.2456 - accuracy: 0.9193 - val_loss: 0.0999 - val_accuracy: 0.9658
Epoch 6/10
121/121 [==============================] - 2s 20ms/step - loss: 0.1433 - accuracy: 0.9515 - val_loss: 0.0613 - val_accuracy: 0.9798
Epoch 7/10
121/121 [==============================] - 2s 20ms/step - loss: 0.1159 - accuracy: 0.9624 - val_loss: 0.0478 - val_accuracy: 0.982

In [30]:
Xtest=Xtest.reshape(-1,pixelsize,pixelsize,9,1)
Xtest.shape

(8556, 25, 25, 9, 1)

In [31]:
ytest=utils.to_categorical(ytest)
ytest.shape

(8556, 9)

In [32]:
Y_pred_test=model.predict(Xtest)
Y_pred_test=np.argmax(Y_pred_test,axis=1)
classification= classification_report(np.argmax(ytest,axis=1),Y_pred_test)
print(classification)

268/268 [==============================] - 2s 4ms/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1326
           1       1.00      1.00      1.00      3730
           2       0.95      0.96      0.95       420
           3       0.99      1.00      0.99       613
           4       0.99      1.00      0.99       269
           5       1.00      1.00      1.00      1006
           6       0.97      0.98      0.98       266
           7       1.00      0.99      0.99       737
           8       0.98      0.95      0.97       189

    accuracy                           0.99      8556
   macro avg       0.98      0.98      0.98      8556
weighted avg       0.99      0.99      0.99      8556

